In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
import random
import os
from newspaper import Article
# pdfminer.six
# import pdfminer
# import pdfminer.high_level
pd.set_option("max_colwidth", 400)
from pandarallel import pandarallel

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.chrome.options import Options
# from icecream import ic
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
import os

chrome_options = Options()
chrome_options.headless = False
chrome_drive = './chromedriver'
os.environ['webdrive.chrome.drive'] = chrome_drive

In [3]:
url = 'https://www.unodc.org/unodc/en/search/index.html#?c=%7B%22filters%22:%5B%7B%22fieldName%22:%22tag%22,%22value%22:%22Press%20Releases%22%7D,%7B%22fieldName%22:%22lang%22,%22value%22:%22en%22%7D%5D,%22match%22:%22%22%7D'
driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_drive)
driver.get(url)

while True:
    try:
        next_find = driver.find_element_by_xpath("//*[contains(text(), 'Load more result')]")
        next_find.click()
        time.sleep(1)
    except:
        break        
html = driver.page_source

In [4]:
driver.quit()

In [16]:
soup_press = BeautifulSoup(html, 'html.parser')
rows_press = soup_press.find_all('div', {'class': 'col ng-scope'})
len(rows_press)

906

In [25]:
rows_press = [{'title':i.h3.text, 'href': i.a['href']}  for i in rows_press]
rows_press

[{'title': 'Unique hybrid 14th UN Congress on Crime Prevention and Criminal Justice closes in Kyoto, Japan',
  'href': '/unodc/press/releases/2021/March/unique-hybrid-14th-un-congress-on-crime-prevention-and-criminal-justice-closes-in-kyoto--japan.html'},
 {'title': "UNODC Executive Director's Message on International Women’s Day",
  'href': '/unodc/press/releases/2021/March/message-unodc-executive-director-on-international-womens-day.html'},
 {'title': 'Kyoto Crime Congress agrees declaration to tackle growing crime threats and promote inclusive COVID-19 recovery',
  'href': '/unodc/press/releases/2021/March/kyoto-crime-congress-agrees-declaration-to-tackle-growing-crime-threats-and-promote-inclusive-covid-19-recovery.html'},
 {'title': 'UNODC Executive Director launches Strategic Vision for Africa 2030  ',
  'href': '/unodc/press/releases/2021/February/unodc-executive-director-launches-strategic-vision-for-africa-2030.html'},
 {'title': 'Share of Children Among Trafficking Victims In

In [26]:
url = 'https://www.unodc.org/unodc/frontpage/index.html'
coverpage = requests.get(url).content
soup_story = BeautifulSoup(coverpage, 'html.parser')
rows_story = soup_story.find_all('a', href=True, title=True)
rows_story = [{'title':i['title'], 'href': i['href']}  for i in rows_story]
rows_story

[{'title': 'To dissociate from trauma, victims of terrorism depict themselves as a faceless body - a new UNODC project helps in their recovery',
  'href': '/unodc/en/frontpage/2021/March/to-dissociate-from-trauma--victims-of-terrorism-depict-themselves-as-a-faceless-body--a-new-unodc-project-helps-in-their-recovery.html'},
 {'title': 'UNODC shares tools and experiences with police and justice experts to respond to gender-based violence against women ',
  'href': '/unodc/en/frontpage/2021/March/unodc-shares-tools-and-experiences-with-police-and-justice-experts-to-respond-to-gender-based-violence-against-women.html'},
 {'title': 'UNODC “Building Networks Behind Prison Walls” Webinar Series continues with special focus on Mental Health in Prisons',
  'href': '/unodc/en/frontpage/2021/March/unodc-building-networks-behind-prison-walls-webinar-series-continues-with-special-focus-on-mental-health-in-prisons.html'},
 {'title': "UNODC's E4J initiative receives Secretary-General 2020 Innovation 

In [27]:
rows = rows_press + rows_story
df = pd.json_normalize(rows)
df.href = df.href.apply(lambda x: 'https://www.unodc.org' + x if x.startswith('/') else x)
df

,title,href
0,"Unique hybrid 14th UN Congress on Crime Prevention and Criminal Justice closes in Kyoto, Japan",https://www.unodc.org/unodc/press/releases/2021/March/unique-hybrid-14th-un-congress-on-crime-prevention-and-criminal-justice-closes-in-kyoto--japan.html
1,UNODC Executive Director's Message on International Women’s Day,https://www.unodc.org/unodc/press/releases/2021/March/message-unodc-executive-director-on-international-womens-day.html
2,Kyoto Crime Congress agrees declaration to tackle growing crime threats and promote inclusive COVID-19 recovery,https://www.unodc.org/unodc/press/releases/2021/March/kyoto-crime-congress-agrees-declaration-to-tackle-growing-crime-threats-and-promote-inclusive-covid-19-recovery.html
3,UNODC Executive Director launches Strategic Vision for Africa 2030,https://www.unodc.org/unodc/press/releases/2021/February/unodc-executive-director-launches-strategic-vision-for-africa-2030.html
4,"Share of Children Among Trafficking Victims Increases, Boys Five Times; COVID-19 Seen Worsening Overall Trend in Human Trafficking, Says UNODC Report",https://www.unodc.org/unodc/press/releases/2021/February/share-of-children-among-trafficking-victims-increases--boys-five-times-covid-19-seen-worsening-overall-trend-in-human-trafficking--says-unodc-report.html
...,...,...
2871,Photo Exhibition on Human Trafficking at UN Headquarters in New York,https://www.unodc.org/unodc/en/frontpage/Photo-Exhibition-on-Human-Trafficking-at-UN-Headquarters.html
2872,World Bank and UNODC to pursue stolen asset recovery,https://www.unodc.org/unodc/en/frontpage/world-bank-and-unodc-to-pursue-stolen-asset-recovery.html
2873,Child trafficking in Ghana,https://www.unodc.org/unodc/en/frontpage/child-trafficking-in-ghana.html
2874,Escopetarra: Instrument of peace,https://www.unodc.org/unodc/en/frontpage/escopetarra-instrument-of-peace.html


In [34]:
df.to_csv('unodc_raw.csv', index=False)

In [2]:
df = pd.read_csv('unodc_raw.csv', header=[0])
df

,title,href
0,"Unique hybrid 14th UN Congress on Crime Prevention and Criminal Justice closes in Kyoto, Japan",https://www.unodc.org/unodc/press/releases/2021/March/unique-hybrid-14th-un-congress-on-crime-prevention-and-criminal-justice-closes-in-kyoto--japan.html
1,UNODC Executive Director's Message on International Women’s Day,https://www.unodc.org/unodc/press/releases/2021/March/message-unodc-executive-director-on-international-womens-day.html
2,Kyoto Crime Congress agrees declaration to tackle growing crime threats and promote inclusive COVID-19 recovery,https://www.unodc.org/unodc/press/releases/2021/March/kyoto-crime-congress-agrees-declaration-to-tackle-growing-crime-threats-and-promote-inclusive-covid-19-recovery.html
3,UNODC Executive Director launches Strategic Vision for Africa 2030,https://www.unodc.org/unodc/press/releases/2021/February/unodc-executive-director-launches-strategic-vision-for-africa-2030.html
4,"Share of Children Among Trafficking Victims Increases, Boys Five Times; COVID-19 Seen Worsening Overall Trend in Human Trafficking, Says UNODC Report",https://www.unodc.org/unodc/press/releases/2021/February/share-of-children-among-trafficking-victims-increases--boys-five-times-covid-19-seen-worsening-overall-trend-in-human-trafficking--says-unodc-report.html
...,...,...
2871,Photo Exhibition on Human Trafficking at UN Headquarters in New York,https://www.unodc.org/unodc/en/frontpage/Photo-Exhibition-on-Human-Trafficking-at-UN-Headquarters.html
2872,World Bank and UNODC to pursue stolen asset recovery,https://www.unodc.org/unodc/en/frontpage/world-bank-and-unodc-to-pursue-stolen-asset-recovery.html
2873,Child trafficking in Ghana,https://www.unodc.org/unodc/en/frontpage/child-trafficking-in-ghana.html
2874,Escopetarra: Instrument of peace,https://www.unodc.org/unodc/en/frontpage/escopetarra-instrument-of-peace.html


In [13]:
from newspaper import Article, Config
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

def extract_text(url):
    if url.endswith('pdf'):
        return 'pdf'
    else:
        try:
            time.sleep(2)
            article = Article(url, config= config)
            article.download()
            article.parse()
            return article.text
        except:
            print('error', url)
            return 'error'

In [4]:
pandarallel.initialize(nb_workers=8)
batch_size = 8*8

i=0
while i<df.shape[0]:
    print(i)
    df.loc[i:i+batch_size-1, 'text']=\
    df.loc[i:i+batch_size-1, 'href'].parallel_apply(extract_text).values
    
    i +=batch_size
    
df.to_csv('unodc_text.csv', index=False)  

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
0
64
error https://www.unodc.org/unodc/press/releases/2018/April/first-advisory-board-of-the-global-judicial-integrity-network-appointed-at-vienna-launch.html
128
192
256
320
384
448
512
error https://www.unodc.org/unodc/press/releases/2012/July/iccwc-launches-wildlife-and-forest-crime-toolkit.html
576
640
704
768
832
error https://www.unodc.org/unodc/press/releases/press_release_2006_05_05.1html
896
960
1024
1088
1152
error https://www.unodc.org/unodc/en/frontpage/2020/February/unodc-launches-hydroponics-and-soap-production-prisoner-rehabilitation-projects-in-namibia.html
error https://www.unodc.org/unodc/en/frontpage/2019/November/from-washington--unodc-launches-guidelines-to-uphold-judicial-integrity-in-the-age-of-social-media.html
1216
1280
error https://www.unodc.org/unodc/en/frontpage/2019/May/the-zorbs--ambassadors-of-peace--justice-and-ru

In [10]:
# df = df[df.text != 'pdf']
df.loc[df.text=='error']

,title,href,text
127,First Advisory Board of the Global Judicial Integrity Network appointed at Vienna launch,https://www.unodc.org/unodc/press/releases/2018/April/first-advisory-board-of-the-global-judicial-integrity-network-appointed-at-vienna-launch.html,error
552,"ICCWC launches wildlife and forest crime toolkit, secures $600,000 to strengthen national enforcement capacities",https://www.unodc.org/unodc/press/releases/2012/July/iccwc-launches-wildlife-and-forest-crime-toolkit.html,error
848,Global Container Control Programme,https://www.unodc.org/unodc/press/releases/press_release_2006_05_05.1html,error
1178,UNODC launches hydroponics and soap production prisoner rehabilitation projects in Namibia,https://www.unodc.org/unodc/en/frontpage/2020/February/unodc-launches-hydroponics-and-soap-production-prisoner-rehabilitation-projects-in-namibia.html,error
1214,"From Washington, UNODC launches guidelines to uphold judicial integrity in the age of social media",https://www.unodc.org/unodc/en/frontpage/2019/November/from-washington--unodc-launches-guidelines-to-uphold-judicial-integrity-in-the-age-of-social-media.html,error
1281,"The Zorbs, Ambassadors of Peace, Justice and Rule of Law, received by UNODC Executive Director",https://www.unodc.org/unodc/en/frontpage/2019/May/the-zorbs--ambassadors-of-peace--justice-and-rule-of-law--received-by-unodc-executive-director.html,error
1293,"To help lower re-offending, UNODC and partners inaugurate two new prison rehabilitation facilities in Zambia",https://www.unodc.org/unodc/en/frontpage/2019/April/to-help-lower-re-offending--unodc-and-partners-inaugurate-two-new-prison-rehabilitation-facilities-in-zambia.html,error
1318,Around the world in three CCPCJs: Model United Nations Conferences tackle SDG16,https://www.unodc.org/unodc/en/frontpage/2019/February/model-united-nations-conferences.html,error
1338,Line up to live up; how sports can help prevent violent extremism in youth,https://www.unodc.org/unodc/en/frontpage/2019/January/line-up-to-live-up-how-sports-can-help-prevent-violent-extremism-in-youth2.html,error
1367,"UNODC's Education for Justice initiative: inspiring, changing, innovating",https://www.unodc.org/unodc/en/frontpage/2018/November/unodcs-education-for-justice-initiative_-inspiring--changing--innovating.html,error


In [12]:
df.loc[(df.text.str.len()<100) & (df.text !='error')]

,title,href,text
18,Standing in solidarity with migrants: Supporting civil society and other stakeholders in responding to the COVID-19 pandemic,https://www.unodc.org/documents/press/releases/2020/July/NW_Statement_for_CSOs_final_version.pdf,pdf
24,The COVID-19 pandemic is an opportunity to reimagine human mobility,https://www.unodc.org/documents/press/releases/network_statement_on_sg_policy_brief_-_final.pdf,pdf
28,Forced returns of migrants must be suspended in times of COVID-19,https://www.unodc.org/documents/press/releases/network_statement_forced_returns_-_13_may_2020.pdf,pdf
33,Wildlife trafficking harms animals and human health: the case of Pangolins,https://www.unodc.org/documents/press/releases/Pangolins_WCR2020_press_release.pdf,pdf
37,LEADERS' STATEMENT on Violence Against Children: A hidden crisis of the COVID-19 pandemic,https://www.unodc.org/documents/Advocacy-Section/Ending_Violence_in_a_time_of_COVID-19_-_Joint_Statement.pdf,pdf
843,NaN,https://www.unodc.org/unodc/press/releases/press_release_2006_05_10_sp.html,
1145,UN Trust Fund for Victims of Human Trafficking Announces 10 NGO Projects Selected for New Emergency Aid Window of 4th Grant Cycle,https://www.unodc.org/unodc/en/frontpage/2020/April/un-trust-fund-for-victims-of-human-trafficking-announces-10-ngo-projects-selected-for-new-emergency-aid-window-of-4th-grant-cycle.html,


In [14]:
pandarallel.initialize(nb_workers=2, progress_bar=True)
df.loc[df.text=='error', 'text']=\
df.loc[df.text=='error', 'href'].parallel_apply(extract_text).values

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


error https://www.unodc.org/unodc/press/releases/2018/April/first-advisory-board-of-the-global-judicial-integrity-network-appointed-at-vienna-launch.html
error https://www.unodc.org/unodc/en/frontpage/2017/May/at-crime-commission--doha-declaration-global-programme-highlights-tangible-ways-in-which-sdgs-are-being-achieved.html
error https://www.unodc.org/unodc/press/releases/2012/July/iccwc-launches-wildlife-and-forest-crime-toolkit.html
error https://www.unodc.org/unodc/en/frontpage/2017/May/latin-america-meeting-marks-first-move-towards-establishing-unodc-supported-global-brand-of-prison-products.html
error https://www.unodc.org/unodc/press/releases/press_release_2006_05_05.1html
error https://www.unodc.org/unodc/en/frontpage/2017/May/innovation-lab-gives-pacific-youth-space-to-develop-creative-solutions-to-corruption.html
error https://www.unodc.org/unodc/en/frontpage/2020/February/unodc-launches-hydroponics-and-soap-production-prisoner-rehabilitation-projects-in-namibia.html
error h

In [15]:
df = df[df.text.str.len()>100].reset_index(drop=True)
df.to_csv('unodc_text.csv', index=False)  

In [16]:
df

,title,href,text
0,"Unique hybrid 14th UN Congress on Crime Prevention and Criminal Justice closes in Kyoto, Japan",https://www.unodc.org/unodc/press/releases/2021/March/unique-hybrid-14th-un-congress-on-crime-prevention-and-criminal-justice-closes-in-kyoto--japan.html,"PRESS RELEASE\n\nUnique hybrid 14th UN Congress on Crime Prevention and Criminal Justice closes in Kyoto, Japan\n\n12 March 2021\n\nKyoto Declaration pledges to strengthen global efforts to combat crime\n\nKYOTO, 12 March (UN Information Service) – The world’s largest gathering on crime prevention and criminal justice wrapped up on Friday with calls for stronger partnerships among different pa..."
1,UNODC Executive Director's Message on International Women’s Day,https://www.unodc.org/unodc/press/releases/2021/March/message-unodc-executive-director-on-international-womens-day.html,"PRESS RELEASE\n\nMessage for International Women’s Day\n\n8 March 2021\n\nKYOTO, 8 March (UN Information Service) – To achieve justice for women, we need more women in justice.\n\nIncreases in women’s representation and leadership in law enforcement and the judiciary have been linked to more investigations into crimes against women, better policing outcomes, and successful victim-centred appro..."
2,Kyoto Crime Congress agrees declaration to tackle growing crime threats and promote inclusive COVID-19 recovery,https://www.unodc.org/unodc/press/releases/2021/March/kyoto-crime-congress-agrees-declaration-to-tackle-growing-crime-threats-and-promote-inclusive-covid-19-recovery.html,"PRESS RELEASE\n\nKyoto Crime Congress agrees declaration to tackle growing crime threats and promote inclusive COVID-19 recovery\n\nKYOTO, 7 March 2021\n\nThe 14th UN Congress on Crime Prevention and Criminal Justice – held in an innovative hybrid format amid pandemic restrictions – opened in Kyoto today with calls for renewing global cooperation against crime to achieve the Sustainable Develo..."
3,UNODC Executive Director launches Strategic Vision for Africa 2030,https://www.unodc.org/unodc/press/releases/2021/February/unodc-executive-director-launches-strategic-vision-for-africa-2030.html,"PRESS RELEASE\n\nUNODC Executive Director launches Strategic Vision for Africa 2030\n\n24 February 2021\n\nThe Executive Director of the United Nations Office on Drugs and Crime (UNODC), Ghada Waly, launched the Office’s Strategic Vision for Africa 2030 at a high-level global online event today.\n\nAgainst the backdrop of the COVID-19 pandemic and its consequences, the Strategic Vision for Afr..."
4,"Share of Children Among Trafficking Victims Increases, Boys Five Times; COVID-19 Seen Worsening Overall Trend in Human Trafficking, Says UNODC Report",https://www.unodc.org/unodc/press/releases/2021/February/share-of-children-among-trafficking-victims-increases--boys-five-times-covid-19-seen-worsening-overall-trend-in-human-trafficking--says-unodc-report.html,"PRESS RELEASE\n\nShare of Children Among Trafficking Victims Increases, Boys Five Times; COVID-19 Seen Worsening Overall Trend in Human Trafficking, Says UNODC Report\n\n2 February 2021\n\nThe share of children among detected trafficking victims has tripled while the share of boys has increased five times in the past 15 years. Girls are mainly trafficked for sexual exploitation, while boys are..."
...,...,...,...
2815,Photo Exhibition on Human Trafficking at UN Headquarters in New York,https://www.unodc.org/unodc/en/frontpage/Photo-Exhibition-on-Human-Trafficking-at-UN-Headquarters.html,"Photo Exhibition on Human Trafficking at UN Headquarters in New York\n\n29 October 2007 - The photography exhibition ""Human Trafficking: Images of Vulnerability""was opened by the UNODC Executive Director Antonio Maria Costa and Mrs. Ban Soon-taek, wife of the Secretary-General of the United Nations today.\n\nFeaturing the photography of special guests Howard G. Buffett, President of the Howard..."
2816,World Bank and UNODC to pursue stolen asset recovery,https://www.unodc.org/unodc/en/

In [29]:
df=pd.read_csv('unodc_text.csv', header=0)
df.columns = ['title', 'url', 'text']
df

,title,url,text
0,Unique hybrid 14th UN Congress on Crime Preven...,https://www.unodc.org/unodc/press/releases/202...,PRESS RELEASE\n\nUnique hybrid 14th UN Congres...
1,UNODC Executive Director's Message on Internat...,https://www.unodc.org/unodc/press/releases/202...,PRESS RELEASE\n\nMessage for International Wom...
2,Kyoto Crime Congress agrees declaration to tac...,https://www.unodc.org/unodc/press/releases/202...,PRESS RELEASE\n\nKyoto Crime Congress agrees d...
3,UNODC Executive Director launches Strategic Vi...,https://www.unodc.org/unodc/press/releases/202...,PRESS RELEASE\n\nUNODC Executive Director laun...
4,Share of Children Among Trafficking Victims In...,https://www.unodc.org/unodc/press/releases/202...,PRESS RELEASE\n\nShare of Children Among Traff...
...,...,...,...
2815,Photo Exhibition on Human Trafficking at UN He...,https://www.unodc.org/unodc/en/frontpage/Photo...,Photo Exhibition on Human Trafficking at UN He...
2816,World Bank and UNODC to pursue stolen asset re...,https://www.unodc.org/unodc/en/frontpage/world...,World Bank and UNODC to Pursue Stolen Asset Re...
2817,Child trafficking in Ghana,https://www.unodc.org/unodc/en/frontpage/child...,Child trafficking in Ghana\n\nBy Raggie Johans...
2818,Escopetarra: Instrument of peace,https://www.unodc.org/unodc/en/frontpage/escop...,Escopetarra: Instrument of peace\n\nBy Norha R...


In [25]:
print(df.text[197])

PRESS RELEASE

UNODC Monitoring Survey reports 14 per cent increase of coca bush cultivation in Bolivia during 2016

La Paz/Vienna, 19 July 2017 - In 2016, coca bush cultivation increased 14 per cent in Bolivia compared to the previous year, according to the latest Coca Crop Monitoring Survey launched today in La Paz by UNODC and the Bolivian Government. In this period, the surface under cultivation increased from 20,200 hectares (ha) to 23,100 ha, following five years of continuous reduction (2011-2015).

According to the same survey, Yungas de La Paz, Trópico de Cochabamba and Norte de La Paz provinces, respectively, accounted for 68, 31 and 1 per cent of the areas under cultivation in the country. Through the use of satellite imaging and field monitoring, a surge of coca bush cultivation was revealed in these three main areas. Compared to 2015, the cultivation in the Yungas de La Paz and Trópico de Cochabamba regions grew from 14,000 to 15,700 ha and from 6,000 to 7,200 ha, respec

In [33]:
df_sample = df.sample(n=20).reset_index(drop=True)
for i,j in zip(df_sample.url, df_sample.text):
    print(i)
    print(j)
    print('\n')

https://www.unodc.org/unodc/press/releases/2015/December/opium-production-in-myanmar-and-the-lao-peoples-democratic-republic-stabilizes-at-high-levels.html
PRESS RELEASE

Opium production in Myanmar and the Lao People's Democratic Republic stabilizes at high levels, says UNODC report

New UNODC survey notes stable opium cultivation in Lao PDR and Myanmar, cautions more needs to be done to address underlying causes of illicit cultivation.

Vienna, 15 December 2015 - Opium poppy cultivation in the Golden Triangle stabilized in 2015 at 55,000 hectares (ha) in Myanmar, and 5,700 ha in the Lao People's Democratic Republic (PDR), according to the Southeast Asia Opium Survey 2015 released today by the United Nations Office on Drugs and Crime (UNODC).

The survey shows that Myanmar remains Southeast Asia's top opium producer - and the world's second largest after Afghanistan. Together, Myanmar and Lao PDR produced between 731 metric tons (mt) and 823 mt of opium, most of which - using the smug

In [30]:
import unicodedata
def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")"))
    return TEXT

df.text = df.text.apply(lambda x: unicodetoascii(x))
df.text = df.text.apply(lambda x: unicodedata.normalize("NFKD", x))
df.text = df.text.apply(lambda x: re.sub(r"http\S+", "", x))
df.text = df.text.apply(lambda x: re.sub("\S+@\S+(?:\.\S+)+",'',x))
df.text = df.text.apply(lambda x: re.sub('\(\+( |-|\d)+\)( |-|\d)+',' ',x))
df.text = df.text.apply(lambda x: re.sub('\+( |-|\d)+',' ',x))

In [32]:
df.text = df.text.apply(lambda x: re.sub('\n\n+','\n\n',x))

In [35]:
df.to_csv('unodc.csv', index=False)

In [42]:
df[df.url.str.contains('2003')]

,title,url,text


In [51]:
df=pd.read_csv('unodc.csv', header=0)
df

,title,url,text
0,"Unique hybrid 14th UN Congress on Crime Prevention and Criminal Justice closes in Kyoto, Japan",https://www.unodc.org/unodc/press/releases/2021/March/unique-hybrid-14th-un-congress-on-crime-prevention-and-criminal-justice-closes-in-kyoto--japan.html,"PRESS RELEASE\n\nUnique hybrid 14th UN Congress on Crime Prevention and Criminal Justice closes in Kyoto, Japan\n\n12 March 2021\n\nKyoto Declaration pledges to strengthen global efforts to combat crime\n\nKYOTO, 12 March (UN Information Service) – The world’s largest gathering on crime prevention and criminal justice wrapped up on Friday with calls for stronger partnerships among different pa..."
1,UNODC Executive Director's Message on International Women’s Day,https://www.unodc.org/unodc/press/releases/2021/March/message-unodc-executive-director-on-international-womens-day.html,"PRESS RELEASE\n\nMessage for International Women’s Day\n\n8 March 2021\n\nKYOTO, 8 March (UN Information Service) – To achieve justice for women, we need more women in justice.\n\nIncreases in women’s representation and leadership in law enforcement and the judiciary have been linked to more investigations into crimes against women, better policing outcomes, and successful victim-centred appro..."
2,Kyoto Crime Congress agrees declaration to tackle growing crime threats and promote inclusive COVID-19 recovery,https://www.unodc.org/unodc/press/releases/2021/March/kyoto-crime-congress-agrees-declaration-to-tackle-growing-crime-threats-and-promote-inclusive-covid-19-recovery.html,"PRESS RELEASE\n\nKyoto Crime Congress agrees declaration to tackle growing crime threats and promote inclusive COVID-19 recovery\n\nKYOTO, 7 March 2021\n\nThe 14th UN Congress on Crime Prevention and Criminal Justice – held in an innovative hybrid format amid pandemic restrictions – opened in Kyoto today with calls for renewing global cooperation against crime to achieve the Sustainable Develo..."
3,UNODC Executive Director launches Strategic Vision for Africa 2030,https://www.unodc.org/unodc/press/releases/2021/February/unodc-executive-director-launches-strategic-vision-for-africa-2030.html,"PRESS RELEASE\n\nUNODC Executive Director launches Strategic Vision for Africa 2030\n\n24 February 2021\n\nThe Executive Director of the United Nations Office on Drugs and Crime (UNODC), Ghada Waly, launched the Office’s Strategic Vision for Africa 2030 at a high-level global online event today.\n\nAgainst the backdrop of the COVID-19 pandemic and its consequences, the Strategic Vision for Afr..."
4,"Share of Children Among Trafficking Victims Increases, Boys Five Times; COVID-19 Seen Worsening Overall Trend in Human Trafficking, Says UNODC Report",https://www.unodc.org/unodc/press/releases/2021/February/share-of-children-among-trafficking-victims-increases--boys-five-times-covid-19-seen-worsening-overall-trend-in-human-trafficking--says-unodc-report.html,"PRESS RELEASE\n\nShare of Children Among Trafficking Victims Increases, Boys Five Times; COVID-19 Seen Worsening Overall Trend in Human Trafficking, Says UNODC Report\n\n2 February 2021\n\nThe share of children among detected trafficking victims has tripled while the share of boys has increased five times in the past 15 years. Girls are mainly trafficked for sexual exploitation, while boys are..."
...,...,...,...
2815,Photo Exhibition on Human Trafficking at UN Headquarters in New York,https://www.unodc.org/unodc/en/frontpage/Photo-Exhibition-on-Human-Trafficking-at-UN-Headquarters.html,"Photo Exhibition on Human Trafficking at UN Headquarters in New York\n\n29 October 2007 - The photography exhibition ""Human Trafficking: Images of Vulnerability""was opened by the UNODC Executive Director Antonio Maria Costa and Mrs. Ban Soon-taek, wife of the Secretary-General of the United Nations today.\n\nFeaturing the photography of special guests Howard G. Buffett, President of the Howard..."
2816,World Bank and UNODC to pursue stolen asset recovery,https://www.unodc.org/unodc/en/f

In [83]:
for i in df[df.text.str.contains('For.*information.*please.*contact')].index:
#     print('\n\n----------')
#     print(i)
    x = re.sub('For.*information.*please.*contact', 'BBBBBB', df.text[i])
    x = x.split('BBBBBB')
#     print(len(x))
    if len(x)==2:
        print(len(x[0]), len(x[1]))
        df.text[i] = x[0]
#         print(x[1])
    if len(x)>2:
        print('Larger than 2', i)
        
#     print('\n\n')

Larger than 2 78
Larger than 2 328


In [81]:
df.text[683] = df.text[683].split('For information, please contact')[0]

In [88]:
df[df.text.str.len()<500]

,title,url,text
1093,Coronavirus disease (COVID-19) - UNODC updates,https://www.unodc.org/unodc/en/frontpage/2020/May/useful-information-about-the-coronavirus-disease-covid-19.html,COVID-19 and the drug supply chain: from production and trafficking to use\n\nSocial Media content\n\nOpinion pieces\n\n22 December 2020 - Judicial Training in Times of COVID-19\n\n23 November 2020 - The Right of Access to Justice in Serbia during the Coronavirus Pandemic\n\n2 November 2020 - The Malaysian Court's Accessibility to Justice in the Time of COVID-19\n\n6 July 2020 - The Supreme Co...
1153,Information about the Coronavirus disease (COVID-19),https://www.unodc.org/unodc/en/frontpage/2020/March/useful-information-about-the-coronavirus-disease-covid-19.html,COVID-19 and the drug supply chain: from production and trafficking to use\n\nSocial Media content\n\nOpinion pieces\n\n22 December 2020 - Judicial Training in Times of COVID-19\n\n23 November 2020 - The Right of Access to Justice in Serbia during the Coronavirus Pandemic\n\n2 November 2020 - The Malaysian Court's Accessibility to Justice in the Time of COVID-19\n\n6 July 2020 - The Supreme Co...
2513,In solidarity with the people of Haiti,https://www.unodc.org/unodc/en/frontpage/2010/January/in-solidarity-with-the-people-of-haiti.html,"In solidarity with the people of Haiti\n\n15 January 2010 - In the wake of the dreadful earthquake in Haiti, UNODC expresses heartfelt condolences and deep sympathy to the people of Haiti, especially the victims of this tragic event. Our thoughts and prayers are with all those affected who may have lost family members, workmates and friends including our United Nations colleagues.\n\nRegular u..."
2684,Executive Director assesses global threats on BBC World Today,https://www.unodc.org/unodc/en/frontpage/executive-director-assesses-global-threats-on-bbc-world-today.html,"Executive Director assesses global threats on BBC World Today\n\nUNODC Executive Director Antonio Maria Costa talks to the BBC World Service about the challenges facing his Office in 2009. Drug trafficking from South America is threatening to destabilize Mexico and Central America, the Caribbean and West Africa, while human trafficking, piracy and other forms of organized crime also loom.\n\nM..."
2695,Watch Antonio Maria Costa on BBC's Hardtalk,https://www.unodc.org/unodc/en/frontpage/unodcs-antonio-maria-costa-on-bbc-hardtalk.html,"Antonio Maria Costa on BBC - Hardtalk\n\n2 December 2008- The Executive Director of UNODC, Antonio Maria Costa will appear today on BBC's Hardtalk. The interviewer is Zeinab Badawi.\n\nIn the interview he discussed Afghanistan, drug trafficking through West Africa, and cannabis.\n\nThis text will be replaced"
2704,Drug trafficking as a security threat in West Africa,https://www.unodc.org/unodc/en/frontpage/drug-trafficking-as-a-security-threat-in-west-africa.html,"Drug trafficking as a security threat in West Africa\n\n28 October 2008 - According to a new UNODC report, a declining US cocaine market and a rising European one appear to have prompted South American cocaine traffickers to make use of low-governance areas in West Africa as transit zones.\n\nPress release on the launch of the report (available in multiple languages).\n\nFull Report English (P..."


In [89]:
df.to_csv('unodc.csv', index=False)